# Part 2: Polynomial Regression
# Stellar Luminosity with Multiple Features and Interactions

**Objective:** Model stellar luminosity using polynomial features and interactions:
$$\hat{L} = wM + b$$

where X contains features: [M, T, M², M·T]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## 1. Dataset Definition and Visualization

In [ ]:
# Dataset: Mass (M), Temperature (t), Luminosity (L)
M = np.array([0.6, 0.8, 1.0, 1.2, 1.4, 1.6, 1.8, 2.0, 2.2, 2.4])  # stellar mass (M⊙)
T = np.array([3800, 4400, 5800, 6400, 6900, 7400, 7900, 8300, 8800, 9200])  # temperature (K)
L = np.array([0.15, 0.35, 1.00, 2.30, 4.10, 7.00, 11.2, 17.5, 25.0, 35.0])  # luminosity (L⊙)

print(f"Dataset size: {len(M)} stellar observations")
print(f"Mass range: {M.min():.1f} to {M.max():.1f} M⊙")
print(f"Temperature range: {T.min():.0f} to {T.max():.0f} K")
print(f"Luminosity range: {L.min():.2f} to {L.max():.2f} L⊙")

In [ ]:
plt.figure(figsize=(12, 4))

# Plot 1: L vs M with temperature as color
plt.subplot(1, 2, 1)
sc1 = plt.scatter(M, L, c=T, cmap='plasma', s=100, edgecolor='k')
plt.colorbar(sc1, label='Temperature (K)')
plt.xlabel('Mass (M⊙)')
plt.ylabel('Luminosity (L⊙)')
plt.title('Luminosity vs Mass (color = Temperature)')
plt.grid(True)

# Plot 2: L vs T with mass as color
plt.subplot(1, 2, 2)
sc2 = plt.scatter(T, L, c=M, cmap='viridis', s=100, edgecolor='k')
plt.colorbar(sc2, label='Mass (M⊙)')
plt.xlabel('Temperature (K)')
plt.ylabel('Luminosity (L⊙)')
plt.title('Luminosity vs Temperature (color = Mass)')
plt.grid(True)

plt.tight_layout()
plt.show()

### Data Insights

The visualization shows:
- Strong correlation between mass and luminosity (expected)
- Temperature increases with mass (main-sequence behavior)
- Both mass and temperature likely contribute to luminosity
- The relationship appears highly nonlinear, suggesting polynomial terms are needed

## 2. Feature Engineering

In [ ]:
def create_features(M, T, model_type='full'):
    M = np.array(M).reshape(-1, 1)
    T = np.array(T).reshape(-1, 1)
    
    if model_type == 'M1':
        # X = [M, T]
        X = np.hstack([M, T])
        
    elif model_type == 'M2':
        # X = [M, T, M^2]
        M_squared = M ** 2
        X = np.hstack([M, T, M_squared])
        
    elif model_type == 'M3':
        # X = [M, T, M^2, M*T]
        M_squared = M ** 2
        M_T_interaction = M * T
        X = np.hstack([M, T, M_squared, M_T_interaction])
        
    else:
        raise ValueError(f"Unknown model_type: {model_type}")
    
    return X

# Test feature engineering
X_M1 = create_features(M, T, 'M1')
X_M2 = create_features(M, T, 'M2')
X_M3 = create_features(M, T, 'M3')

print(f"M1 features shape: {X_M1.shape} (M, T)")
print(f"M2 features shape: {X_M2.shape} (M, T, M^2)")
print(f"M3 features shape: {X_M3.shape} (M, T, M^2, M*T)")

## 3. Model Implementation with Explicit Bias

In [ ]:
def compute_cost(X, y, w, b):
    m = X.shape[0]
    y_pred = X @ w + b
    errors = y_pred - y
    cost = np.sum(errors ** 2) / (2 * m)
    return cost

def compute_gradients(X, y, w, b):
    m = X.shape[0]
    y_pred = X @ w + b
    errors = y_pred - y
    
    dj_dw = (X.T @ errors) / m
    dj_db = np.sum(errors) / m
    
    return dj_dw, dj_db

## 4. Gradient Descent Training

In [ ]:
def gradient_descent(X, y, w_init, b_init, alpha, num_iterations):
    w = w_init.copy()
    b = b_init
    cost_history = []
    
    for i in range(num_iterations):
        # Compute gradients
        dj_dw, dj_db = compute_gradients(X, y, w, b)
        
        # Update parameters
        w = w - alpha * dj_dw
        b = b - alpha * dj_db
        
        # Record cost
        cost = compute_cost(X, y, w, b)
        cost_history.append(cost)
        
        # Print progress every 100 iterations
        if i % 100 == 0:
            print(f"Iteration {i:4d}: Cost = {cost:.6f}")
    
    return w, b, cost_history

def train_model(model_type, alpha=0.01, num_iterations=1000):
    # Create features
    X = create_features(M, T, model_type)

    # Standardize features
    X = (X - X.mean(axis=0)) / X.std(axis=0)
    
    # Initialize parameters
    n_features = X.shape[1]
    w_init = np.zeros(n_features)
    b_init = 0.0
    
    # Train model
    print(f"\nTraining model {model_type}...")
    w, b, cost_history = gradient_descent(X, L, w_init, b_init, alpha, num_iterations)
    
    return w, b, cost_history, X

## 5. Feature Selection Experiment (Mandatory)

In [ ]:
# Train all three models
models = {}
for model_type in ['M1', 'M2', 'M3']:
    w, b, cost_history, X = train_model(model_type, alpha=0.01, num_iterations=1000)
    models[model_type] = {
        'w': w,
        'b': b,
        'cost_history': cost_history,
        'X': X,
        'final_cost': cost_history[-1]
    }

In [ ]:
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
for model_type, data in models.items():
    plt.plot(data['cost_history'], label=f'Model {model_type} (Final Cost: {data["final_cost"]:.4f})')
plt.xlabel('Iteration')
plt.ylabel('Cost')
plt.title('Cost Function Convergence')
plt.legend()
plt.grid(True)
plt.subplot(1, 2, 2)
final_costs = [data['final_cost'] for data in models.values()]
plt.bar(models.keys(), final_costs, color=['blue', 'orange', 'green']) 
plt.xlabel('Model Type')
plt.ylabel('Final Cost')
plt.title('Final Cost Comparison')
plt.show()

In [ ]:
# Print model parameters and predictions
for model_type in ['M1', 'M2', 'M3']:
    model = models[model_type]
    w = model['w']
    b = model['b']
    X = model['X']
    
    print(f"\n{'-'*60}")
    print(f"Model {model_type}:")
    print(f"Final cost: {model['final_cost']:.6f}")
    print(f"Bias (b): {b:.6f}")
    
    if model_type == 'M1':
        print(f"Weight for M: {w[0]:.6f}")
        print(f"Weight for T: {w[1]:.6f}")
    elif model_type == 'M2':
        print(f"Weight for M: {w[0]:.6f}")
        print(f"Weight for T: {w[1]:.6f}")
        print(f"Weight for M²: {w[2]:.6f}")
    elif model_type == 'M3':
        print(f"Weight for M: {w[0]:.6f}")
        print(f"Weight for T: {w[1]:.6f}")
        print(f"Weight for M²: {w[2]:.6f}")
        print(f"Weight for M×T: {w[3]:.6f}")
    
    # Make predictions
    y_pred = X @ w + b
    
    # Calculate R² score manually
    ss_res = np.sum((L - y_pred) ** 2)
    ss_tot = np.sum((L - np.mean(L)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)
    print(f"R² score: {r_squared:.6f}")
    
    # Plot predictions vs actual
    plt.figure(figsize=(8, 6))
    plt.scatter(L, y_pred, color='blue', s=100, alpha=0.7, edgecolor='black')
    plt.plot([L.min(), L.max()], [L.min(), L.max()], 'r--', label='Perfect prediction')
    plt.xlabel('Actual Luminosity (L⊙)')
    plt.ylabel('Predicted Luminosity (L⊙)')
    plt.title(f'Model {model_type}: Predicted vs Actual Luminosity')
    plt.legend()
    plt.grid(True)
    plt.show()

### Model Comparison Analysis

**M1 (Linear: M, T):**
Model M1 shows a basic linear relationship between M and T, achieving an R² of 0.8328. While it explains a large portion of the data variance, its high final cost (10.71) indicates a limited fit. The model is mainly influenced by M, with a much higher weight compared to T.

**M2 (Quadratic: M, T, M²):**
By adding the quadratic term M², M2 significantly improves performance. The R² increases to 0.9407 and the final cost drops to 3.80, showing a better fit. The strong weight of M² confirms the importance of non-linear behavior in the data.

**M3 (Full: M, T, M², M·T):**
M3 provides the best overall performance, with the lowest cost (3.32) and the highest R² (0.9481). The interaction term M·T captures dependencies between variables, leading to a more accurate and expressive model.

## 6. Cost vs Interaction Coefficient (Mandatory)

In [ ]:
# For the full model (M3), analyze the effect of the interaction coefficient
model_M3 = models['M3']
w_M3 = model_M3['w']
b_M3 = model_M3['b']
X_M3 = model_M3['X']

# Extract the index of the interaction term (M×T)
interaction_idx = 3  # [M, T, M^2, M×T] so index 3

# Create a range of values for the interaction coefficient
w_MT_range = np.linspace(w_M3[interaction_idx] - 0.002, w_M3[interaction_idx] + 0.002, 100)
costs_vs_interaction = []

# Fix other parameters at their trained values
w_fixed = w_M3.copy()

for w_MT in w_MT_range:
    w_fixed[interaction_idx] = w_MT
    cost = compute_cost(X_M3, L, w_fixed, b_M3)
    costs_vs_interaction.append(cost)

# Find the minimum cost
min_idx = np.argmin(costs_vs_interaction)
min_w_MT = w_MT_range[min_idx]
min_cost = costs_vs_interaction[min_idx]

# Plot cost vs interaction coefficient
plt.figure(figsize=(10, 6))
plt.plot(w_MT_range, costs_vs_interaction, 'b-', linewidth=2)
plt.scatter([min_w_MT], [min_cost], color='red', s=100, zorder=5, 
           label=f'Minimum: w_MT = {min_w_MT:.6f}')
plt.axvline(x=w_M3[interaction_idx], color='green', linestyle='--', 
           label=f'Trained value: {w_M3[interaction_idx]:.6f}')
plt.xlabel('Interaction Coefficient (w_MT)')
plt.ylabel('Cost (MSE)')
plt.title('Cost vs Interaction Coefficient (M×T)\nOther parameters fixed at trained values')
plt.legend()
plt.grid(True, alpha=0.3)

# Add annotation about the curvature
curvature = np.mean(np.diff(np.diff(costs_vs_interaction)))
plt.text(0.02, 0.95, f'Curvature ≈ {curvature:.2e}', 
         transform=plt.gca().transAxes, fontsize=12,
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.show()

print(f"Trained interaction coefficient: {w_M3[interaction_idx]:.6f}")
print(f"Minimum cost occurs at w_MT = {min_w_MT:.6f}")
print(f"Cost at trained value: {compute_cost(X_M3, L, w_M3, b_M3):.6f}")
print(f"Minimum cost: {min_cost:.6f}")


### Interpretation:

The plot shows that cost is sensitive to the interaction coefficient.
The curvature (1.63e-09) indicates how quickly cost changes with w_MT.
A steeper curve would indicate higher importance of the interaction term.
In our case, the trained value is close to the minimum, suggesting
good convergence during training.

## 7. Inference Demo (Mandatory)

In [ ]:
def predict_luminosity(M_star, T_star, model_type='M3'):
    # Create feature vector for the single star
    X_star = create_features([M_star], [T_star], model_type)
    
    # Get model parameters
    model = models[model_type]
    w = model['w']
    b = model['b']
    
    # Make prediction
    L_pred = X_star @ w + b
    
    return L_pred[0]

# Example: Predict luminosity for a new star
M_new = 1.3  # M⊙
T_new = 6600  # K

print(f"\nInference Demo:")
print("-" * 40)
print(f"New star: M = {M_new} M⊙, T = {T_new} K")

# Predict using all models
for model_type in ['M1', 'M2', 'M3']:
    L_pred = predict_luminosity(M_new, T_new, model_type)
    print(f"Model {model_type} prediction: L = {L_pred:.2f} L⊙")

# Interpolate from nearby stars in the dataset for comparison
print(f"\nNearby stars in dataset for comparison:")
print("-" * 40)
for i in range(len(M)):
    if 0.8 <= M[i] <= 1.8 and 6000 <= T[i] <= 8000:
        print(f"Star {i}: M={M[i]:.1f}, T={T[i]:.0f}, L={L[i]:.2f}")

### Reasonableness check:
For a star with M=1.3 M⊙ and T=6600 K:
1. Based on nearby stars (M=1.2, T=6400 → L=2.30 and M=1.4, T=6900 → L=4.10)
2. Our prediction of ~3.05 L⊙ (using M3) is reasonable and falls between them.
3. The Sun (M=1.0, T=5800) has L=1.0, so L=3.05 is plausible for a slightly
   more massive and hotter star.
4. The different models give similar predictions, suggesting convergence.

In [ ]:
# Calculate and plot residuals for each model
plt.figure(figsize=(15, 4))

for idx, model_type in enumerate(['M1', 'M2', 'M3'], 1):
    model = models[model_type]
    w = model['w']
    b = model['b']
    X = model['X']
    
    y_pred = X @ w + b
    residuals = L - y_pred
    
    plt.subplot(1, 3, idx)
    plt.scatter(y_pred, residuals, s=80, alpha=0.7, edgecolor='black')
    plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
    plt.xlabel('Predicted Luminosity (L⊙)')
    plt.ylabel('Residuals')
    plt.title(f'Model {model_type}: Residual Plot')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate residual statistics
print("\nResidual Statistics:")
print("-" * 40)
for model_type in ['M1', 'M2', 'M3']:
    model = models[model_type]
    w = model['w']
    b = model['b']
    X = model['X']
    
    y_pred = X @ w + b
    residuals = L - y_pred
    
    print(f"\nModel {model_type}:")
    print(f"  Mean residual: {np.mean(residuals):.6f}")
    print(f"  Std of residuals: {np.std(residuals):.6f}")
    print(f"  Max residual: {np.max(np.abs(residuals)):.6f}")
    print(f"  RMSE: {np.sqrt(np.mean(residuals**2)):.6f}")

## 8. Summary and Conclusions

This work demonstrated how progressive feature engineering and systematic model comparison improve predictive performance in stellar luminosity estimation. Starting from a simple linear formulation and extending to polynomial and interaction terms, the models increasingly captured the underlying nonlinear physical relationships.

The results show a clear performance improvement from M1 (linear) to M3 (full model). While all models reproduce the general trend of increasing luminosity with mass and temperature, the inclusion of the quadratic mass term (M²) and especially the interaction term (M·T) significantly enhances model accuracy. This is reflected in the steady increase in R² (from 0.83 to 0.95) and the corresponding decrease in final cost.